### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from pandas import json_normalize

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url= (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    +f"&fq={filter_query}&sort={sort}&qfl={field_list}"
)
articles = requests.get(query_url).json()


In [4]:
# Create an empty list to store the reviews
articles_list = []

# loop through pages 0-19
for page in range(0, 20):
    # create query with a page number
    query_url = f"{query_url}&page={str(page+1)}"

    # API results show 10 articles at a time

    
    # Make a "GET" request and retrieve the JSON
    articles = requests.get(query_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for article in articles["response"]["docs"]:
            articles_list.append(article)
    
    
    
        

        # Print the page number that had no results then break from the loop
    except:
        print(f"You ended at page {page}")
        
        break

    # Print the page that was just retrieved
    print(f"Page {page} was successfully added")

Page 0 was successfully added


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(articles_list[:5], indent=4))

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
articles_list_df = pd.json_normalize(articles_list)
articles_list_df

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"

# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
import re

def extracted_title(headline_main):
    extracted_title = re.search(r'\u2018(.*?)\u2019 Review', headline_main)
    if extracted_title:
        return extracted_title.group(1)
    else:
        return None

articles_list_df['Title'] = articles_list_df['headline.main'].apply(lambda x: extracted_title(x))
articles_list_df.head()

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column

def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords


# Fix the "keywords" column by converting cells from a list to a string
articles_list_df['keywords'] = articles_list_df['keywords'].apply(extract_keywords)
articles_list_df.head()

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = articles_list_df['Title'].to_list()
title_list

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
movies = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
x=0
while x < 50: 
   

# Loop through the titles

    # Check if we need to sleep before making a request
    if x <= 50:
        time.sleep(10)
    else:

    # Add 1 to the request counter
        x += 1
    
    # Perform a "GET" request for The Movie Database
        movie_url = f"{url}{title_list[x]}{tmdb_key_string}"
        response = requests.get(movie_url)

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
        try:

        # Get movie id
            movie_id = x['id']

        # Make a request for a the full movie details
            details_url = "'https://api.themoviedb.org/3/movie/"

        # Execute "GET" request with url
            full_url = f"{details_url}{movie_id}{tmdb_key_string}"
            full_details = requests.get(full_url)

        except:
            print("Movie not found.")
        
        # Extract the genre names into a list
            for genre in full_details['genres']:
                genre_list = full_details.append(full_details['genres']['name'])



        # Extract the spoken_languages' English name into a list
            for languages in full_details['spoken_languages']:
                language_list = full_details.append(full_details['spoken_languages']['name'])

        # Extract the production_countries' name into a list
            for country in full_details['production_countries']:
                country_list = full_details.append(full_details['production_countries']['name'])

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
            new_columns_dict = {k: v for k, v in genre_list},{k: v for k, v in language_list},{k: v for k, v in country_list}
        
        # Print out the title that was found
            print(f'{full_details['original_title']} was found.')



In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data


In [ ]:
# Convert the results to a DataFrame


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


In [ ]:
# Export data to CSV without the index
